In [3]:
from Network_PER import *
from DriveSimTrace import *

In [2]:
#시뮬레이션 학습 코드
EXP_COUNT = 1
n_runs = 10000
rwd_components = 3
train_scores = np.zeros([EXP_COUNT,n_runs//100])
train_losses = np.zeros([EXP_COUNT,n_runs//100])

for exps in range(EXP_COUNT):
    wins = 0
    per = True
    lrate = 0.0002
    print("learning rate:", lrate)
    agent = Agent(lr=lrate, gamma=0.99, n_actions=5, batch_size=64, epsilon=1.0, input_dims=[1,7], per_on=per)

    sim = DriveSimulator()
    epsilons = []
    avg_score = 0.0
    avg_loss = 0.0
    
    replace = np.array([])
    replace = np.append(replace, np.asarray(range(100,10001,100)))
    #replace = np.append(replace, np.asarray(range(3000,5001,100)))
    #replace = np.append(replace, np.asarray(range(6000,10001,500)))
    print(replace)

    for i in range(n_runs):
        if i in replace:
            agent.set_next_weight()

        over = False
        sim.reset(frame_rate=3000)

        state = sim.get_sim_state()
        while not over:
            action, pred, pred_C = agent.choose_action(state)
            #print(action, pred)
            state_, stpRwd, sim_over = sim.step(action, pred_C)
            over = sim_over
            
            agent.store_transition(state, action, stpRwd, state_, over, pred)
            state = state_

        Loss = agent.learn(exps)
        #overestimation detection


        agent.epsilon = round((0.9 - 0.8 * (float(i)/float(n_runs))),3)
        #scores.append(sim.agtRwd)
    
        avg_score += np.sum(sim.agtRwd)
        avg_loss += np.sum(Loss)
        if sim.episode_count % 100 == 0:
            avg_score /= 100
            avg_loss /= 100
            print('Episode', sim.episode_count, ', average score(last 100 episodes):', avg_score, \
                'average loss:', avg_loss, 'wins:', sim.win_count, 'eps:', agent.epsilon)

            train_scores[exps, i//100] = avg_score
            train_losses[exps, i//100] = avg_loss
            avg_score = 0.0
            avg_loss = 0.0
            epsilons.append(agent.epsilon)

    agent.save_model(f'./models/q_20230220({exps})')
np.savetxt(f'20220220_scores.csv', train_scores, delimiter=",")
np.savetxt(f'20220220_losses.csv', train_losses, delimiter=",")

learning rate: 0.0002
Use Prioritized Sampling: True
[  100.   200.   300.   400.   500.   600.   700.   800.   900.  1000.
  1100.  1200.  1300.  1400.  1500.  1600.  1700.  1800.  1900.  2000.
  2100.  2200.  2300.  2400.  2500.  2600.  2700.  2800.  2900.  3000.
  3100.  3200.  3300.  3400.  3500.  3600.  3700.  3800.  3900.  4000.
  4100.  4200.  4300.  4400.  4500.  4600.  4700.  4800.  4900.  5000.
  5100.  5200.  5300.  5400.  5500.  5600.  5700.  5800.  5900.  6000.
  6100.  6200.  6300.  6400.  6500.  6600.  6700.  6800.  6900.  7000.
  7100.  7200.  7300.  7400.  7500.  7600.  7700.  7800.  7900.  8000.
  8100.  8200.  8300.  8400.  8500.  8600.  8700.  8800.  8900.  9000.
  9100.  9200.  9300.  9400.  9500.  9600.  9700.  9800.  9900. 10000.]
Episode 100 , average score(last 100 episodes): -5.8 average loss: 1.5092919336725026 wins: 21 eps: 0.894
q_next weight set!
Episode 200 , average score(last 100 episodes): -6.6 average loss: 1.0772863848041743 wins: 38 eps: 0.888
q_nex

In [ ]:
#시뮬레이션 학습 코드 (추가 학습)
EXP_COUNT = 1
n_runs = 10000
rwd_components = 4
train_scores = np.zeros([EXP_COUNT,n_runs//100])
train_losses = np.zeros([EXP_COUNT,n_runs//100])

for exps in range(EXP_COUNT):
    wins = 0
    per = True
    lrate = 0.0005
    print("learning rate:", lrate)
    agent = Agent(lr=lrate, gamma=0.999, n_actions=5, batch_size=64, epsilon=1.0, input_dims=[4,5], per_on=per)
    
    model = f'./models/q_20230214(0)'
    agent.load_model(model)

    sim = DriveSimulator()
    epsilons = []
    avg_score = 0.0
    avg_loss = 0.0
    
    for i in range(n_runs):
        over = False
        sim.reset(frame_rate=3000)

        state = sim.get_sim_state()
        while not over:
            action, pred, pred_C = agent.choose_action(state)
            #print(action, pred)
            state_, stpRwd, sim_over = sim.step(action, pred_C)
            over = sim_over
            
            agent.store_transition(state, action, stpRwd, state_, over, pred)
            state = state_
        
        Loss = agent.learn(exps)
        #overestimation detection


        agent.epsilon = round((1.0 - (float(i)/float(n_runs))),3)
        #scores.append(sim.agtRwd)
    
        avg_score += np.sum(sim.agtRwd)
        avg_loss += np.sum(Loss)
        if sim.episode_count % 100 == 0:
            avg_score /= 100
            avg_loss /= 100
            print('Episode', sim.episode_count, ', average score(last 100 episodes):', avg_score, \
                'average loss:', avg_loss, 'wins:', sim.win_count, 'eps:', agent.epsilon)

            train_scores[exps, i//100] = avg_score
            train_losses[exps, i//100] = avg_loss
            avg_score = 0.0
            avg_loss = 0.0
            epsilons.append(agent.epsilon)

    agent.save_model(f'./models/q_20230216({exps})')
np.savetxt(f'20220216_scores.csv', train_scores, delimiter=",")
np.savetxt(f'20220216_losses.csv', train_losses, delimiter=",")

In [ ]:

def save_graph(date, label):
    scores = np.genfromtxt(date + '_scores.csv', delimiter=',')
    losses = np.genfromtxt(date + '_losses.csv', delimiter=',')

    # Plot Score
    plt.title(label + ' (score)')
    for i in range(EXP_COUNT):
        plt.plot(scores[i], 'b')
        plt.plot(8.0 * np.ones_like(scores[i]), 'gray') #Score 상한선
        plt.plot(-5.0 * np.ones_like(scores[i]), 'gray') #Score 하한선
        plt.show()
    #plt.savefig(date + '_score.png')
    #plt.show()

    for i in range(EXP_COUNT):
        plt.title(label + ' (loss)')
        #Plot Loss
        plt.plot(losses[i], 'orange')
        plt.plot(1.5 * np.ones_like(losses[i]), 'gray') #Loss 상한선
        plt.plot(np.zeros_like(losses[i]), 'gray') #Loss 하한선
        plt.show()
    #plt.savefig(date + '_loss.png')
    #plt.show()

save_graph(date = '20221209', label = 'test1')